In [1]:
#Imports
#====================
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#====================

data = pd.read_csv('final_train_data.csv')

data.head()

,Unnamed: 0,Country Name,Country Code,Year,Balance,Inflation,GDP,Exports,Trade
0,0,Senegal,SEN,1983,-3.043829e+08,9.274213,-5.326393,6.180000e+08,59.224298
1,1,Togo,TGO,2009,-1.766692e+08,1.855275,3.510297,9.030260e+08,76.242542
2,2,Pakistan,PAK,1992,-1.876092e+09,10.057085,7.705898,7.351000e+09,34.489440
3,3,Nepal,NPL,1988,-2.714506e+08,11.815310,7.696809,1.900000e+08,24.978422
4,5,Malta,MLT,2016,7.269132e+08,1.604548,5.516617,3.017299e+09,83.781471


#### Filling in NaN Values

In [2]:
year_to_features_given_country = dict()
for idx,row in data.iterrows():
    row.to_dict()
    try:
        year_to_features_given_country[row['Country Code']].append(row)
    except KeyError:
        year_to_features_given_country[row['Country Code']] = [row]
for country, data_list in year_to_features_given_country.items():
    temp = dict()
    for _ in data_list:
        temp[_['Year']] = [_['Inflation'],_['GDP'],_['Exports'],_['Trade']]
    year_to_features_given_country[country] = temp

In [3]:
train_data = []
for idx,row in data.iterrows():
    row.to_dict()
    line = []
    line.append(row['Country Code'])
    line.append(row['Balance'])
    line.append(row['Year'])
    for idx,feature in enumerate(['Inflation','GDP','Exports','Trade']):
        if np.isnan(row[feature]):
            min_year = min(year_to_features_given_country[row['Country Code']].keys())
            curr_year = row['Year']
            s=False
            for c in range(curr_year,min_year,-1):
                try:
                    imputed_feature = year_to_features_given_country[row['Country Code']][c][idx]
                    if not np.isnan(imputed_feature):
                        line.append(imputed_feature)
                        print('imputed',imputed_feature,'for',feature,'in the year',curr_year)
                        s=True
                        break
                except KeyError:
                    continue
            if s==False:
                line.append("Remove")
        else:
            line.append(row[feature])
    if "Remove" not in line:
        train_data.append(line)

imputed 14.27570645 for Inflation in the year 2014
imputed -6.881302064 for GDP in the year 2014
imputed 429.3684626 for Trade in the year 2014
imputed 2.962802421 for Inflation in the year 2016
imputed 6.000194324 for GDP in the year 2016
imputed 58.70044877 for Trade in the year 2016
imputed 14.27570645 for Inflation in the year 2011
imputed -6.881302064 for GDP in the year 2011
imputed 4933844549.0 for Exports in the year 2016
imputed 107.4659439 for Trade in the year 2016
imputed 4933844549.0 for Exports in the year 2015
imputed 107.4659439 for Trade in the year 2015
imputed 14.27570645 for Inflation in the year 2012
imputed -6.881302064 for GDP in the year 2012
imputed 429.3684626 for Trade in the year 2012
imputed 2.965962733 for Inflation in the year 1991
imputed -10.0496666 for GDP in the year 1991
imputed 18.25071664 for Inflation in the year 2015
imputed -62.07591958 for GDP in the year 2015
imputed 77.79973568 for Trade in the year 2015
imputed 14.27570645 for Inflation in t

In [4]:
train_data = pd.DataFrame(train_data,columns=['Country Code','Balance','Year','Inflation','GDP','Exports','Trade'])
train_data.head()

,Country Code,Balance,Year,Inflation,GDP,Exports,Trade
0,SEN,-3.043829e+08,1983,9.274213,-5.326393,6.180000e+08,59.224298
1,TGO,-1.766692e+08,2009,1.855275,3.510297,9.030260e+08,76.242542
2,PAK,-1.876092e+09,1992,10.057085,7.705898,7.351000e+09,34.489440
3,NPL,-2.714506e+08,1988,11.815310,7.696809,1.900000e+08,24.978422
4,MLT,7.269132e+08,2016,1.604548,5.516617,3.017299e+09,83.781471


In [5]:
print("Checking for NaN values in Dataset")
print(train_data.isnull().any())
print("Original Size:",len(data))
print("New Size:",len(train_data))

Checking for NaN values in Dataset
Country Code    False
Balance         False
Year            False
Inflation       False
GDP             False
Exports         False
Trade           False
dtype: bool
Original Size: 4506
New Size: 4240


In [6]:
column_to_min_max = dict()
for column in train_data.columns[2:]:
    print(column)
    print("Max:",max(train_data[column]))
    print("Min:",min(train_data[column]))
    column_to_min_max[column]={'min':min(train_data[column]),'max':max(train_data[column])}
    print('===================================')

Year
Max: 2016
Min: 1961
Inflation
Max: 12338.66161
Min: -29.69106658
GDP
Max: 34.49999951
Min: -62.07591958
Exports
Max: 2340000000000.0
Min: 2000000.0
Trade
Max: 957.7840112
Min: 7.920909083


In [7]:
def min_max_normalize(value,column):
    return (value-column_to_min_max[column]['max'])/(column_to_min_max[column]['max']-column_to_min_max[column]['min'])
train_data['norm_year'] = train_data['Year'].apply(lambda x:min_max_normalize(x,'Year'))
train_data['norm_gdp'] = train_data['GDP'].apply(lambda x:min_max_normalize(x,'GDP'))
train_data['norm_trade'] = train_data['Trade'].apply(lambda x:min_max_normalize(x,'Trade'))
train_data['norm_inflation'] = train_data['Inflation'].apply(lambda x:min_max_normalize(x,'Inflation'))
train_data['norm_exports'] = train_data['Exports'].apply(lambda x:min_max_normalize(x,'Exports'))

In [8]:
df = train_data[['Balance','norm_year','norm_gdp','norm_trade','norm_inflation','norm_exports']]
rs = np.random.RandomState(0)
corr = df.corr()
corr.style.background_gradient()

,Balance,norm_year,norm_gdp,norm_trade,norm_inflation,norm_exports
Balance,1,0.0123001,0.0186439,0.0744837,0.00119983,-0.0145227
norm_year,0.0123001,1,-0.0328948,0.0790919,-0.0478494,0.205763
norm_gdp,0.0186439,-0.0328948,1,0.0700877,-0.0660294,-0.0295597
norm_trade,0.0744837,0.0790919,0.0700877,1,-0.0242342,-0.0322988
norm_inflation,0.00119983,-0.0478494,-0.0660294,-0.0242342,1,-0.0220469
norm_exports,-0.0145227,0.205763,-0.0295597,-0.0322988,-0.0220469,1


In [9]:
df_normed = train_data[['Country Code','Balance','norm_year','norm_gdp','norm_trade','norm_inflation','norm_exports']]

In [10]:
df_normed.head()

,Country Code,Balance,norm_year,norm_gdp,norm_trade,norm_inflation,norm_exports
0,SEN,-3.043829e+08,-0.600000,-0.412384,-0.945989,-0.996850,-0.999737
1,TGO,-1.766692e+08,-0.127273,-0.320884,-0.928072,-0.997449,-0.999615
2,PAK,-1.876092e+09,-0.436364,-0.277441,-0.972029,-0.996786,-0.996859
3,NPL,-2.714506e+08,-0.509091,-0.277535,-0.982042,-0.996644,-0.999920
4,MLT,7.269132e+08,0.000000,-0.300110,-0.920135,-0.997470,-0.998711


In [11]:
df_normed.to_excel("normed_train_data.xlsx")

In [12]:
import pickle
with open("feature_normalization_dict.pickle",'wb') as f:
    pickle.dump(column_to_min_max,f)